**TREE-CODE**

This algorithm does an approximation on the force computation. 
Given N particles in a volume V, tree-code devides teh volume iteratively into cubes up to the point each one contains only 1 particle. 
When the force acting over one particle i is computed, particles sufficiently far away from it are grouped in one point (the center ??) and their mass is sum up in total mass M. This is equal to take only the first order of the multipolar expansion (monopole term).

Computational cost: $N\log(N)$

Leaf opening criterion: $\theta = r_{cell}/d < \theta_{crit}$ 

$\theta_{crit}$ is the accuracy parameter (<<1 radiant)

Gravitational softening: to avoid huge scatterings we set the force different from newtonian one if the particles get to much close:

$|f_{ij}|=\frac{Gm_im_j}{|r_i-r_j|^2+\epsilon^2}$

does not diverge: it's like the mass of each particle has been smoothed over a finite volume.

Time integration: LEAP-FROG method. $t_i -> t_{i+1}$

$v_{i+\frac{1}{2}}=v_i+(\frac{h}{2})a_i$

$r_{i+1}=r_i+hv_{i+\frac{1}{2}}$

$v_{i+1}=v_{i+\frac{1}{2}}+(\frac{h}{2})a_i$

h is NOT ADAPTIVE, it's set at imput. We will set $h<<t_{dyn}$

$\epsilon = 10^{-4}(V/N)^{1/3}$

**INTERNAL UNITS**

nbody_sh1 works with internal units. $G_{iu}$ is set equal to 1. We are free to express masses and distances in arbitrary units, as far as velocities and times are consistent. In order to see that, notice that $\frac{GM}{rv^2}$ is adimensional. 

$$\frac{M_{iu}}{r_{iu}v_{iu}^2}=\frac{G_{cgs}M_{cgs}}{r_{cgs}v_{cgs}^2}$$

Velocities in internal units will be related to physical velocities by:

$$v_{iu}=\sqrt{\frac{r_{cgs}}{G_{cgs}M_{cgs}}}v_{cgs}$$

We need to tranform time units in internal units as well. $\frac{rt}{v}$ is adimensional. 

$$\frac{t_{iu}v_{iu}}{r_{iu}}=\frac{t_{cgs}v_{cgs}}{r_{cgs}}$$

$$t_{iu}=\frac{r_{iu}}{r_{cgs}}\sqrt{\frac{r_{cgs}}{G_{cgs}M_{cgs}}}t_{cgs}$$

**HOMOLOGOUS COLLAPSE**

$$\ddot{r}=-\frac{Gm(r)}{r^2}$$

$$\dot{r}\ddot{r}=-\frac{Gm(r)}{r^2}\dot{r}$$

$$\int_{0}^t \dot{r}\ddot{r} dt = -Gm(r_0)\int_{0}^t \frac{\dot{r}}{r^2} dt$$

HOMOLOGOUS COLLAPSE (show it?): $m(r(t))=m(r_0)$ at every $t$. 

$$\frac{1}{2}\dot{r(t)}^2=Gm(r_0)\left(\frac{1}{r(t)}-\frac{1}{r_0}\right)$$

$$K(t)=Gm_{star}\frac{4\pi}{3}\rho r_0^3\left(\frac{1}{r(t)}-\frac{1}{r_0}\right)$$

Given initial $r_0$, what is $r(t)$ during the homologous collapse? Integrating $\dot{r(t)}$:


$$-\sqrt{r_0}\left( \sqrt{r(t)(r_0-r(t))}+r_0\arccos{\sqrt{\frac{r(t)}{r_0}}}\right)=-\sqrt{\frac{8\pi}{3}\rho r_0^3} t$$ holds for $t<t_{free fall}=\sqrt{\frac{3\pi}{32G\rho}}$, $r(t)<r_0 \forall t$

The equation is not analytical: it has to be solved numerically. Define function f:

$$f_{r_0,t}(r)=-\sqrt{r_0}\left( \sqrt{r(r_0-r)}+r_0\arccos{\sqrt{\frac{r}{r_0}}}\right)+\sqrt{\frac{8\pi}{3}\rho r_0^3} t$$

At every $t$, and for every $r_0$, $r(t)$ is found with bisection method: extremes of integration are $10^{-6}$ (function not defined in 0) and $r_0$ (NOT R!) 


**GENERATING AN HOMOGENEUS SPHERE**

$$dV = (r^2 dr)(\sin\theta d\theta)(d \phi)$$

$$p(r) = \frac{1}{M} 4 \pi r^2 \rho_0 $$

$$P(r) =  \int_0^r p(r) dr = \frac{4\pi \rho_0}{3M} r^3   $$
 
$$\int_0^R p(r) dr = 1$$

$$r(P) = \left(\frac{3MP}{4\pi \rho_0}\right)^{1/3}$$

$$p(\theta) = \frac{\sin\theta}{2}$$

$$P(\theta) = \int_0^\theta p(\theta) = \frac{-\cos\theta+1}{2}$$

$$\theta(P)=\arccos(1-2P)$$

$$\int_0^\pi p(\theta) = 1$$

$$p(\phi) = \frac{1}{2\pi}$$

$$P(\phi) = \frac{\phi}{2\pi}$$

$$\phi(P) = 2\pi P $$

In [1]:
import numpy as np
import random 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.gridspec as gridspec
from IPython.display import HTML 
import scipy

import subprocess 
mpl.rcParams['animation.embed_limit'] = 2**128


In [2]:
c_cgs = 2.99792458 * 10**10 #cm/s
G_cgs = 6.67259 * 10**-8 #G in cgs
M_sun = 1.9891 * 10**33 #solar mass in g
R_sun = 6.9598 * 10**10 #solar radius in cm 
M_earth = 5.976 * 10**27 #earth mass in g
R_earth = 6.378 * 10**8 #earth radius in cm
ly = 9.463 * 10**17 #light year in cm
parsec = 3.086 * 10**18 #parsec in cm
AU = 1.496 * 10**13 #astronomical unit in cm

def v_IU(M_cgs, r_cgs, v_cgs):
    return np.sqrt(r_cgs/(G_cgs*M_cgs))*v_cgs

def c_IU(M_cgs, r_cgs):
    return np.sqrt(r_cgs/(G_cgs*M_cgs))*c_cgs

def t_IU(M_cgs, r_cgs, t_cgs):
    return t_cgs/(np.sqrt(r_cgs/(G_cgs*M_cgs))*r_cgs)

print("We choose 1 solar mass as mass unit and 1 parsec unit as distance unit.")
print("1 Myr expressed in internal units is: %f" % (t_IU(M_sun,parsec,3.156*10**13)))
print("1 time unit expressed in cgs is: %f Myrs" % (1/t_IU(M_sun,parsec,3.156*10**13)))
print("Light speed expressed in internal units is: %E" % (c_IU(M_sun,parsec)))
print("Sun rest mass energy in internal units is: %E" %(1*c_IU(M_sun,parsec)))
print("Sun potential energy (3/5GM^2/R = %E erg) in internal units is: %E" %((3/5)*G_cgs*M_sun**2/R_sun, (3/5)/(R_sun/parsec)))

We choose 1 solar mass as mass unit and 1 parsec unit as distance unit.
1 Myr expressed in internal units is: 0.067069
1 time unit expressed in cgs is: 14.910128 Myrs
Light speed expressed in internal units is: 4.571336E+06
Sun rest mass energy in internal units is: 4.571336E+06
Sun potential energy (3/5GM^2/R = 2.275947E+48 erg) in internal units is: 2.660421E+07


In [3]:
time_conv = 0.067069 #1 Myr in internal units
energy_conv = 1/4571335.907452 #sun rest mass energy in internal units

class Star:
    def __init__(self, m, x, v): #x is the cartesian coordinate vector, v the cartesian velocity
        self.m = m
        self.position = np.array(x)
        self.velocity = np.array(v)
        self.r = np.sqrt(x[0]**2+x[1]**2+x[2]**2)
        self.kinetic_energy = energy_conv*(0.5)*self.m*(v[0]**2+v[1]**2+v[2]**2) #expressed in sun rest mass energy?
 
class Homogeneus_Galaxy:
    def __init__(self, N, rho, m, code, filename="file"): #N is the number of stars, rho is the density, m the single star mass
        self.N = int(N)
        self.rho = rho
        self.m = m
        self.M = self.N*self.m
        self.R = (3*self.M/(4*np.pi*self.rho))**(1/3)
        self.filename = filename
        self.code=code
        print("Galaxy of %i stars of mass %.2f. Density is %.2f solar masses for parsec cube, radius is %.2f parsecs, total mass %.2f solar masses" %(self.N, self.m, self.rho, self.R, self.M))
        
        #random population of our homogeneus galaxy
        points_pol=[] #polar coordinates of points
        points_cart=[] #cartesian coordinates of the points
        stars=[] #stars
        self.colors_hom=[] #colors of stars to check homologous collapse
        self.newcolors=[]
        for i in range(self.N):
            u = random.uniform(0,1) #no need to pass seed to random method. By default the random number generator uses the current system time
            v = random.uniform(0,1)
            w = random.uniform(0,1)
            points_pol.append([(3*self.M*u/(4*np.pi*self.rho))**(1/3),np.arccos(1-2*v),2*np.pi*w])  
        r = np.array(points_pol)[:,0]
        for point in points_pol:
            points_cart.append([point[0]*np.sin(point[1])*np.cos(point[2]),point[0]*np.sin(point[1])*np.sin(point[2]),point[0]*np.cos(point[1])])
        for i in range(len(points_cart)):
            stars.append(Star(self.m, points_cart[i], [0,0,0]))
            if r[i]<self.R/4:
                self.colors_hom.append("blue")
            elif r[i]>=self.R/4 and r[i]<self.R/2:
                self.colors_hom.append("green")
            elif r[i]>=self.R/2 and r[i]<3*self.R/5:
                self.colors_hom.append("red")
            elif r[i]>=3*self.R/5 and r[i]<3*self.R/4:
                self.colors_hom.append("orange")
            else:
                self.colors_hom.append("purple")
        #stars.sort(key=lambda star: star.r) #sorts star array in ascending distance order: very important for color gradient
        self.initial_star_collection = np.array(stars)

        #writes free fall time
        self.free_fall_time = np.sqrt(3*np.pi/(32*self.rho))
        print("Free fall time is %f in internal units, %f Myr" %(self.free_fall_time, self.free_fall_time*time_conv))

        #writes initial conditions in file for treecode
        if code=="treecode":
            initial_conditions = open("%s.in" %filename, "w") 
            initial_conditions.write(str(self.N)+"\n") #number of bodies
            initial_conditions.write(str(3)+"\n") #dimensions 
            initial_conditions.write(str(0.)+"\n") #initial time
            for star in self.initial_star_collection:
                initial_conditions.write(str(self.m)+"\n")   
            for star in self.initial_star_collection: #works with sorted array
                initial_conditions.write(str(star.position[0])+"\t"+str(star.position[1])+"\t"+str(star.position[2])+"\n")
            for star in self.initial_star_collection: #works with sorted array
                initial_conditions.write(str(star.velocity[0])+"\t"+str(star.velocity[1])+"\t"+str(star.velocity[2])+"\n")
            initial_conditions.close()
        #writes initial conditions in file for nbody_sh1
        if code=="nbody_sh1":
            initial_conditions = open("%s.in" %filename, "w") 
            initial_conditions.write(str(self.N)+"\n")
            initial_conditions.write(str(0)+"\n")
            for star in self.initial_star_collection: #works with sorted array
                initial_conditions.write(str(self.m)+"\t"+str(star.position[0])+"\t"+str(star.position[1])+"\t"+str(star.position[2])+"\t"+str(star.velocity[0])+"\t"+str(star.velocity[1])+"\t"+str(star.velocity[2])+"\n")
            initial_conditions.close()


        #makes array usefull for later
        self.t = []
        self.evolution_star_collections = []
        if code=="treecode":
            print("GALAXY NOT EVOLVED. Run:     ./treecode in=%s.in out=%s.out dtime=%f theta=%f eps=%f tstop=%f dtout=%f > %s_logfile   to cover at least 10 free fall time with time step 1/1000 of dynamical time" %(filename,filename,0.001*self.free_fall_time,0.1,10**-4 *self.N/(4*np.pi*self.R**3/3),10*self.free_fall_time, 0.01*self.free_fall_time,filename))
        if code=="nbody_sh1":
            print("GALAXY NOT EVOLVED. Run:  ./nbody_sh1 -d 0.03 -e 1.0 -o 0.01 -t %f < initial_conditions.in > output.out  to cover at least 1 free fall time." %self.free_fall_time)

    ##########################################################################
    
    def evolve(self):
        if self.code=="treecode":
            print("ATTENTION: BE SURE TREECODE HAS RUN")
            #opens the output file and writes star's positions array for each time in t
            output = open("%s.out" %self.filename, "r").readlines()
            #makes array of array of star
            for i in range(int(len(output)/(3*self.N+3))):
                self.t.append(float(output[i*(3*self.N+3)+2])) #time in internal units
                star_collection = []
                for j in range(self.N): 
                    p = output[i*(3*self.N+3)+3+self.N+j]
                    #float(output[i*(self.N+2)+(2+j)].split()[1])
                    v = output[i*(3*self.N+3)+3+2*self.N+j]
                    star_collection.append(Star(self.m,[float(p.split()[0]),float(p.split()[1]),float(p.split()[2])],[float(v.split()[0]),float(v.split()[1]),float(v.split()[2])]))
                self.evolution_star_collections.append(star_collection)
        if self.code=="nbody_sh1":
            print("ATTENTION: BE SURE NBODY_SH1 HAS RUN")
            #opens the output file and writes star's positions array for each time in t
            output = open("%s.out" %self.filename, "r").readlines()
            #makes array of array of star
            for i in range(int(len(output)/(self.N+2))):
                self.t.append(float(output[i*(self.N+2)+1])) #time in internal units
                star_collection = []
                for j in range(self.N): 
                    star_collection.append(Star(self.m, [float(output[i*(self.N+2)+(2+j)].split()[1]),float(output[i*(self.N+2)+(2+j)].split()[2]),float(output[i*(self.N+2)+(2+j)].split()[3])], [float(output[i*(self.N+2)+(2+j)].split()[4]),float(output[i*(self.N+2)+(2+j)].split()[5]),float(output[i*(self.N+2)+(2+j)].split()[6])]))
                self.evolution_star_collections.append(star_collection)
            
    #######################################################################

    
    def real_potential(self, cart_r, frame):
        if frame ==0:
        #if time == 0:
            potential = 0
            for star in self.initial_star_collection: 
                d = np.sqrt((star.position[0]-cart_r[0])**2+(star.position[1]-cart_r[1])**2+(star.position[2]-cart_r[2])**2)
                if d.any() != 0:
                    potential += -star.m/d
        else:
            #print("ATTENTION: BE SURE NBODY_SH1 HAS RUN")
            #i = 0
            #for t in self.t:
            #    while(t<time): i+=1 #AGGIUNGERE UN CONTROLLO!
            #print(i)
            potential = 0
            for star in self.evolution_star_collections[frame]: 
                d = np.sqrt((star.position[0]-cart_r[0])**2+(star.position[1]-cart_r[1])**2+(star.position[2]-cart_r[2])**2)
                if d.any()!= 0:
                    potential += -star.m/d
        return potential
            

    def analytic_homogeneus_sphere_potential(self, r):
        if(r<self.R):
            return -2*np.pi*self.rho*(self.R**2-(1/3)*r**2)
        else:
            return -((4/3)*np.pi*self.rho*self.R**3)/r


    def plot_galaxy_time_zero(self):

        fig = plt.figure(figsize=(15, 15))
        gs = gridspec.GridSpec(2, 2)
        #fig.title("Homogeneus galaxy at time zero")
        ax1 = fig.add_subplot(gs[0,0], projection='3d')
        ax2 = fig.add_subplot(gs[0,1], projection='3d')
        ax3 = fig.add_subplot(gs[1,:])
        
        potential_col = []
        error_pot = []
        for star in self.initial_star_collection:
            potential_col.append(self.real_potential(star.position,0)*energy_conv)
            #relative error on analytic potential
            error_pot.append((self.real_potential(star.position,0)-self.analytic_homogeneus_sphere_potential(star.r))/self.analytic_homogeneus_sphere_potential(star.r))

        p = ax1.scatter([star.position[0] for star in self.initial_star_collection], [star.position[1] for star in self.initial_star_collection], [star.position[2] for star in self.initial_star_collection], c=potential_col, cmap="autumn")
        e = ax2.scatter([star.position[0] for star in self.initial_star_collection], [star.position[1] for star in self.initial_star_collection], [star.position[2] for star in self.initial_star_collection], c=error_pot, cmap = "winter") #my_cmap_r = reverse_colourmap(my_cmap)
        ax1,ax2.set_xlim(-self.R, self.R)
        ax1,ax2.set_ylim(-self.R, self.R)
        ax1,ax2.set_zlim(-self.R, self.R)
        ax1,ax2.set_xlabel('X [parsec]')
        ax1,ax2.set_ylabel('Y [parsec]')
        ax1,ax2.set_zlabel('Z [parsec]')
        fig.colorbar(p, ax=ax1, label="Real Potential [units of solar potential energy]", orientation="horizontal")
        fig.colorbar(e, ax=ax2, label="Relative error (real-analytic)/analytic", orientation="horizontal")

        #show the potential as a function of r only
        range = 2*self.R
        step = 0.01*self.R
        r_arr= np.linspace(0, range, int(range/step+1))
        real_potential=[] #4 potentials (4 random angles)
        an_potential=[]
        theta = random.uniform(0,np.pi)
        phi = random.uniform(0,2*np.pi)
        for r in r_arr:
            cart_r = [r*np.sin(theta)*np.cos(phi),r*np.sin(theta)*np.sin(phi),r*np.cos(theta)]
            real_potential.append(self.real_potential(cart_r, 0))
            an_potential.append(self.analytic_homogeneus_sphere_potential(r))

        ax3.plot(r_arr,real_potential, label="real potential for random $\\theta =$ "+str(theta)+" $\phi =$ "+str(phi))
        ax3.plot(r_arr,an_potential, label="analytic potential")
        ax3.set_xlabel("r [parsec]")
        ax3.set_ylabel("potential [units of solar potential energy]")
        ax3.legend()
        plt.show()

    def potential_evolution(self,speed_up):

        fig = plt.figure(figsize=(15, 15))
        gs = gridspec.GridSpec(3, 2)
        #fig.title("Homogeneus galaxy at time zero")
        ax1 = fig.add_subplot(gs[0,0])
        ax2 = fig.add_subplot(gs[0,1])
        ax3 = fig.add_subplot(gs[1,0])
        ax4 = fig.add_subplot(gs[1,1])
        ax5 = fig.add_subplot(gs[2,0])
        ax6 = fig.add_subplot(gs[2,1])
        
        r_range = 2*self.R
        step = 0.01*self.R
        r_arr = np.linspace(0, r_range, int(r_range/step+1))
        angles = np.array([[random.uniform(0,np.pi),random.uniform(0,2*np.pi)],[random.uniform(0,np.pi),random.uniform(0,2*np.pi)],[random.uniform(0,np.pi),random.uniform(0,2*np.pi)],[random.uniform(0,np.pi),random.uniform(0,2*np.pi)],[random.uniform(0,np.pi),random.uniform(0,2*np.pi)],[random.uniform(0,np.pi),random.uniform(0,2*np.pi)]])
        r_cart = []
        for i in range(6):
            r_cart.append([r_arr*np.sin(angles[i,0])*np.cos(angles[i,1]),r_arr*np.sin(angles[i,0])*np.sin(angles[i,1]),r_arr*np.cos(angles[i,0])])
        
        real_potential=[] 
        for i in range(6):
            real_potential.append(self.real_potential(r_cart[i], 0))

        ax1.plot(r_arr,real_potential[0]) #label="real potential for random $\\theta =$ "+str(angles[0,0])+" $\phi =$ "+str(angles[,0])
        ax2.plot(r_arr,real_potential[1])
        ax3.plot(r_arr,real_potential[2])
        ax4.plot(r_arr,real_potential[3])
        ax5.plot(r_arr,real_potential[4])
        ax6.plot(r_arr,real_potential[5])
        fig.suptitle('t = %f Myr, %f free_fall_time' %(self.t[0]*time_conv, self.t[0]/self.free_fall_time), fontsize=30) 
        
        def anim(frame):
            #print(t[frame*speed_up])
            #time = self.t[frame*speed_up]
            real_potential=[] 
            for i in range(6):
                real_potential.append(self.real_potential(r_cart[i], frame*speed_up))
            fig.suptitle('t = %f Myr, %f free_fall_time' %(self.t[frame*speed_up]*time_conv, self.t[frame*speed_up]/self.free_fall_time), fontsize=30) 

            #ax1.cla()
            #ax2.cla()
            #ax3.cla()
            #ax4.cla()
            #ax5.cla()
            #ax6.cla()
            ax1.plot(r_arr,real_potential[0]) #label="real potential for random $\\theta =$ "+str(angles[0,0])+" $\phi =$ "+str(angles[,0])
            ax2.plot(r_arr,real_potential[1])
            ax3.plot(r_arr,real_potential[2])
            ax4.plot(r_arr,real_potential[3])
            ax5.plot(r_arr,real_potential[4])
            ax6.plot(r_arr,real_potential[5])

        ani=animation.FuncAnimation(fig, anim, frames=int(len(self.evolution_star_collections)/speed_up)-1, interval=100) 
 
        display(HTML(ani.to_jshtml()))


        plt.close()


    ######################################################################################################################

    

    def kinetic_evolution_hist(self,speed_up):
        print("ATTENTION: HAVE YOU CALLED evolve()?")

        #first frame 
        kin = []
        for star in self.initial_star_collection:
            kin.append(star.kinetic_energy) #energy_conv already in star.kinetic_energy

        fig,ax = plt.subplots()
        bins = int(self.N/10)
        energy_range = 10
        hist = plt.hist(kin,bins)
        ax.set_xlabel('kinetic energy [units of solar potential energy]') 
        ax.set_ylabel('number of stars') 
        ax.set_xlim(0,energy_range*((3/5)*self.M**2/self.R)/self.N)
        ax.set_ylim(0,self.N)

        def update_hist(frame):
            kin = []
            for star in self.evolution_star_collections[frame*speed_up]:
                kin.append(star.kinetic_energy)
            plt.cla()
            plt.hist(kin,bins)
            ax.set_xlim(0,energy_range*((3/5)*self.M**2/self.R)/self.N)
            ax.set_ylim(0,self.N)
            fig.suptitle('t = %f Myr, %f free_fall_time' %(self.t[frame*speed_up]*time_conv, self.t[frame*speed_up]/self.free_fall_time)) 

        ani = animation.FuncAnimation(fig, update_hist, frames=int(len(self.evolution_star_collections)/speed_up)-1, interval=200)

        display(HTML(ani.to_jshtml()))
        ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/homogeneus_collapse/%s_kinetic_hist.html" %self.filename, writer="html")
        #ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/treecode/%s_kinetic_hist.gif" %self.filename, writer="pillow")
        plt.close()





    
        #checks evolution of energies
    def kinetic_evolution_plot(self,speed):
        if speed>len(self.evolution_star_collections): speed = 1

        fig,ax = plt.subplots(figsize=(6,5), tight_layout=True) #figsize=(10, 10), , tight_layout=True

        K_lim = energy_conv*self.M*self.m/self.R #maximum energy a particle of the galaxy reaches during a truly homogeneus collapse
        r_initial = np.linspace(10**-5,self.R,100) #starting point can't be 0. Function f is not defined in 0
        f = lambda rt, r_initial, t : -np.sqrt(r_initial)*(np.sqrt(rt*(r_initial-rt))+r_initial*np.arccos(np.sqrt(rt/r_initial)))+np.sqrt(8*np.pi*self.rho*r_initial**3 / 3)*t
        
        def animate_scatters(frame): 
            ax.cla()
            r = []
            kin = []
            for star in self.evolution_star_collections[frame*int(speed)]:
                r.append(star.r)
                kin.append(star.kinetic_energy)
            ax.scatter(r, kin, s=1., c='b', marker='o', label="real points") 

            #analytic curve
            t = self.t[frame*int(speed)]
            if(t<self.free_fall_time):
                r_of_t = []
                for r0 in r_initial:
                    r_of_t.append(scipy.optimize.bisect(f, 10**-6, r0, args=(r0, t), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)) 
                kin_of_t = []
                for i in range(len(r_initial)):
                    kin_of_t.append(energy_conv*self.m*(4*np.pi*self.rho*r_initial[i]**3 /3)*(1/r_of_t[i] - 1/r_initial[i]))

                ax.plot(r_initial,kin_of_t, linewidth=2, c="r", label = "Analytic prediction" ) 
            
            ax.set_xlim(0,2*self.R)
            ax.set_ylim(0,K_lim)
            ax.set_xlabel('r [parsecs]') 
            ax.set_ylabel('kinetic energy [units of solar potential energy]') 
            ax.legend(loc='upper right') #, bbox_to_anchor=(1, 0.5)
            
            plt.title('t = %f Myr, %f free_fall_time' %(t*time_conv, t/self.free_fall_time), fontsize=10) 

        ani = animation.FuncAnimation(fig, animate_scatters, frames=int(len(self.evolution_star_collections)/int(speed))-1, interval=200)

        display(HTML(ani.to_jshtml()))
        ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/homogeneus_collapse/%s_kinetic_animation.html" %self.filename, writer="html")
        #ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/treecode/%s_kinetic_animation.gif" %self.filename, writer="pillow")
        plt.close()
    



























    
    def collapse_animation(self, speed): 
        if speed>len(self.evolution_star_collections): speed = 1
        
        #converts list in array, usefull for animation
        #initial frame
        position_of_nth_star = []
        for star in self.initial_star_collection:
                position_of_nth_star.append([star.position[0],star.position[1],star.position[2]])  
        #evolved frames. First frame is the initial one
        positions_at_given_frame = [position_of_nth_star]
        for frame in self.evolution_star_collections:
            position_of_nth_star = []
            for star in frame:
                position_of_nth_star.append([star.position[0],star.position[1],star.position[2]])
            positions_at_given_frame.append(np.array(position_of_nth_star))
        P = np.array(positions_at_given_frame)
       
        #create figure and set axes
        fig = plt.figure(figsize=(15, 15))
        ax = fig.add_subplot(projection='3d')
        ax.set_xlim3d(-2*self.R, 2*self.R)
        ax.set_ylim3d(-2*self.R, 2*self.R)
        ax.set_zlim3d(-2*self.R, 2*self.R)
        ax.set_xlabel('X [parsec]')
        ax.set_ylabel('Y [parsec]')
        ax.set_zlabel('Z [parsec]')
        ax.set_aspect("equal") 
        # Provide starting angle for the view.
        #ax.view_init(25, 10)

        color = []
        c1 = np.array(mpl.colors.to_rgb('#D9480F')) #blue '#1f77b4'
        c2 = np.array(mpl.colors.to_rgb('#FFA94D')) #green
        #c3 = np.array(mpl.colors.to_rgb('green')) 
        for i in range(self.N):
            color.append(mpl.colors.to_hex((1-i/self.N)*c1 + (i/self.N)*c2))
        
        scatters = [ ax.scatter(P[0][i,0:1], P[0][i,1:2], P[0][i,2:], color=self.colors_hom[i]) for i in range(self.N) ]   #color=self.colors[i]
       
        def animate_scatters(frame): 
            t = self.t[frame*int(speed)]
            for i in range(self.N): 
                scatters[i]._offsets3d = (P[frame*int(speed)][i,0:1], P[frame*int(speed)][i,1:2], P[frame*int(speed)][i,2:])
            plt.title('t = %f Myr, %f free_fall_time' %(t*time_conv, t/self.free_fall_time), fontsize=10) 
            return scatters

        ani=animation.FuncAnimation(fig, animate_scatters, frames=int(len(P)/speed)-1, interval=200) 
        display(HTML(ani.to_jshtml()))
        ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/homogeneus_collapse/%s_collapse_animation.html" %self.filename, writer="html")
        #ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/treecode/%s_collapse_animation.gif" %self.filename, writer="pillow")
        plt.close()

    


In [51]:
small_galaxy = Homogeneus_Galaxy(100,1,0.1,"treecode","small_galaxy")

Galaxy of 100 stars of mass 0.10. Density is 1.00 solar masses for parsec cube, radius is 1.34 parsecs, total mass 10.00 solar masses
Free fall time is 0.542701 in internal units, 0.036398 Myr
GALAXY NOT EVOLVED. Run:     ./treecode in=small_galaxy.in out=small_galaxy.out dtime=0.000543 theta=0.100000 eps=0.001000 tstop=5.427009 dtout=0.005427 > small_galaxy_logfile   to cover at least 10 free fall time with time step 1/1000 of dynamical time


In [52]:
small_galaxy.evolve()

ATTENTION: BE SURE TREECODE HAS RUN


In [55]:
medium_galaxy = Homogeneus_Galaxy(1000,1,0.1,"treecode","medium_galaxy")

Galaxy of 1000 stars of mass 0.10. Density is 1.00 solar masses for parsec cube, radius is 2.88 parsecs, total mass 100.00 solar masses
Free fall time is 0.542701 in internal units, 0.036398 Myr
GALAXY NOT EVOLVED. Run:     ./treecode in=medium_galaxy.in out=medium_galaxy.out dtime=0.000543 theta=0.100000 eps=0.001000 tstop=5.427009 dtout=0.005427 > medium_galaxy_logfile   to cover at least 10 free fall time with time step 1/1000 of dynamical time


In [4]:
other_medium_galaxy = Homogeneus_Galaxy(2000,1,0.1,"treecode","other_medium_galaxy")

Galaxy of 2000 stars of mass 0.10. Density is 1.00 solar masses for parsec cube, radius is 3.63 parsecs, total mass 200.00 solar masses
Free fall time is 0.542701 in internal units, 0.036398 Myr
GALAXY NOT EVOLVED. Run:     ./treecode in=other_medium_galaxy.in out=other_medium_galaxy.out dtime=0.000543 theta=0.100000 eps=0.001000 tstop=5.427009 dtout=0.005427 > other_medium_galaxy_logfile   to cover at least 10 free fall time with time step 1/1000 of dynamical time
